In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext("local", "pyspark-shell")

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# Regression

## One-Hot Encoding

Categorical variables can be converted to indexed numerical values to be used in a model. But in general this is not sufficient for a regression model since number have no objective meaning to make arithmatics on the index. You need to conver the index values into a format in which you can perform meaningful mathematical operations (exploding). You can exploit this by converting the data into a sparse format. Sparse representation simply records the column numbers and value for the non-zero values also you can drop one of the columns. The process of creating dummy variables is called "One-Hot Encoding" because only one of the columns created is ever active or "hot". 

    vector = [1,0,0,0,0,7,0,0]
    SparseVector(8, [0, 5], [1, 7])

### Encoding flight origin

The org column in the flights data is a categorical variable, it needs to be one-hot encoded before it can be used in a regression model.

In [8]:
flights = spark.read.csv("flights.csv", sep=",", header=True, inferSchema=True, nullValue="NA")

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="org", outputCol="org_idx")
indexer_model = indexer.fit(flights)
flights = indexer_model.transform(flights)

flights.show()

+---+---+---+-------+------+---+----+------+--------+-----+-------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|org_idx|
+---+---+---+-------+------+---+----+------+--------+-----+-------+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| null|    2.0|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|    0.0|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|    1.0|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|    0.0|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| null|    0.0|
|  5|  2|  1|     UA|   704|SFO| 550|  7.98|     102|    2|    1.0|
|  7|  2|  6|     AA|   380|ORD| 733| 10.83|     135|   54|    0.0|
|  1| 16|  6|     UA|  1477|ORD|1440|   8.0|     232|   -7|    0.0|
|  1| 22|  5|     UA|   620|SJC|1829|  7.98|     250|  -13|    4.0|
| 11|  8|  1|     OO|  5590|SFO| 158|  7.77|      60|   88|    1.0|
|  4| 26|  1|     AA|  1144|SFO|1464| 13.25|     210|  -10|    1.0|
|  4| 25|  0|     AA|   321|ORD| 978| 13.75|    

In [56]:
from pyspark.ml.feature import OneHotEncoder

onehot = OneHotEncoder(inputCols=["org_idx"], outputCols=["org_dummy"])
onehot = onehot.fit(flights)
flights_onehot = onehot.transform(flights)
flights_onehot.select("org", "org_idx", "org_dummy").distinct().sort("org_idx").show()

+---+-------+-------------+
|org|org_idx|    org_dummy|
+---+-------+-------------+
|ORD|    0.0|(7,[0],[1.0])|
|SFO|    1.0|(7,[1],[1.0])|
|JFK|    2.0|(7,[2],[1.0])|
|LGA|    3.0|(7,[3],[1.0])|
|SJC|    4.0|(7,[4],[1.0])|
|SMF|    5.0|(7,[5],[1.0])|
|TUS|    6.0|(7,[6],[1.0])|
|OGG|    7.0|    (7,[],[])|
+---+-------+-------------+



Notice the "OGG"'s org_dummy

## Regression

Regression is for prediction numeric values. MSE (mean squared error) is used to find the mest modeul by minimizing a loss function (min residues). 

Classes expect to find the target data in a column called "label". If it not named label you have to specify the target column. 

It's useful to have a single number which summarizes the performance of a model. 

The intercept is the value predicted by the model when all predictors are zero. 

The slope represents how rapidly the model changes when that predictor changes. The coefficients attribute gives access to those values. There's a coefficient for each of the predictors. 

### Flight duration model: Just distance

In [66]:
from pyspark.sql.functions import round
flights_km = flights.withColumn("km", round(flights.mile * 1.60934, 0)).drop("mile")

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=  ["km"], outputCol="features")
output = assembler.transform(flights_km)

flight_train, flight_test = output.randomSplit([0.8, 0.2], 13)

from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

regression = LinearRegression(labelCol="duration").fit(flight_train)

predictions = regression.transform(flight_test)
predictions.select("duration", "prediction").show(5, False)
RegressionEvaluator(labelCol="duration").evaluate(predictions)

+--------+------------------+
|duration|prediction        |
+--------+------------------+
|385     |359.2475231916086 |
|379     |345.7063569756502 |
|130     |133.66228488999982|
|230     |238.81435774017962|
|64      |72.99180832464427 |
+--------+------------------+
only showing top 5 rows



17.023657662904867

### Interpreting the coefficients

In [69]:
inter = regression.intercept
print(inter)

coefs = regression.coefficients
print(coefs)

minutes_per_km = regression.coefficients[0]

avg_speed = 60 / minutes_per_km

print(avg_speed)

44.39649642725724
[0.07564897327351065]
793.1370037643285


### Flight duration model: Adding origin airport


In [80]:
flights_m = flights_onehot.withColumn("km", round(flights_onehot.mile * 1.60934, 0)).drop("mile")

assembler = VectorAssembler(inputCols=  ["km","org_dummy"], outputCol="features")
output = assembler.transform(flights_m)

flight_train, flight_test = output.randomSplit([0.8, 0.2], 13)

from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

regression = LinearRegression(labelCol="duration").fit(flight_train)

predictions = regression.transform(flight_test)
predictions.select("duration", "prediction").show(5, False)
RegressionEvaluator(labelCol="duration").evaluate(predictions)

+--------+------------------+
|duration|prediction        |
+--------+------------------+
|385     |377.7863176281456 |
|379     |364.4835911703198 |
|130     |132.02676932478195|
|230     |259.4738007518407 |
|64      |72.42460944111582 |
+--------+------------------+
only showing top 5 rows



11.012777355873434

### Interpreting coefficients

The values for km and org_dummy have been assembled into features, which has eight columns with sparse representation. 0 is km and categorical variables are indexed from 1 to 8.

In [102]:
output.select("features").show(5, False)

+----------------------+
|features              |
+----------------------+
|(8,[0,3],[3465.0,1.0])|
|(8,[0,1],[509.0,1.0]) |
|(8,[0,2],[542.0,1.0]) |
|(8,[0,1],[1989.0,1.0])|
|(8,[0,1],[415.0,1.0]) |
+----------------------+
only showing top 5 rows



In [100]:
print(regression.coefficients)
print()

avg_speed_hour = 60 / regression.coefficients[0]
print(avg_speed_hour)

inter = regression.intercept # Average minutes on ground at OGG
print(inter)

avg_ground_jfk = inter + regression.coefficients[3]
print(avg_ground_jfk)

avg_ground_lga = inter + regression.coefficients[4]
print(avg_ground_lga)

[0.07431690758561862,28.413496002892245,20.400159408670234,52.560025885941116,46.64807165450426,18.29933170052044,15.504667559982039,17.58313343757769]

807.3532921277102
15.919322370859733
68.47934825680085
62.56739402536399


## Bucketing & Engineering

The largest improvements in ML model performance are ofthen achieved by carefully manipulating features.

It's often convenient to convert a continuous variable, like age or height, into discrete values. This can be done by assigning values to buckets or bins with well defined boundaries. Resulting categorical variable is ofthen a more powerful predictor than the original continuous variable. To use them in a regression model, they first need to be one-hot encoded.

There are many other approaches to engineering new features. Applying arithmetic operations to one or more columns to create new features. Powerful new features are ofthen discovered through trial and error. Selecting only the relevant predictors is important.

### Bucketing departure time

In [105]:
from pyspark.ml.feature import Bucketizer, OneHotEncoder

buckets = Bucketizer(splits=[0,3,6,9,12,15,18,21,24], inputCol="depart", outputCol="depart_bucket")
bucketed = buckets.transform(flights)
bucketed.select("depart", "depart_bucket").show(5)

onehot = OneHotEncoder(inputCols=["depart_bucket"], outputCols=["depart_dummy"])
flights_onehot = onehot.fit(bucketed).transform(bucketed)
flights_onehot.select("depart", "depart_bucket", "depart_dummy").show(5)

+------+-------------+
|depart|depart_bucket|
+------+-------------+
|  9.48|          3.0|
| 16.33|          5.0|
|  6.17|          2.0|
| 10.33|          3.0|
|  8.92|          2.0|
+------+-------------+
only showing top 5 rows

+------+-------------+-------------+
|depart|depart_bucket| depart_dummy|
+------+-------------+-------------+
|  9.48|          3.0|(7,[3],[1.0])|
| 16.33|          5.0|(7,[5],[1.0])|
|  6.17|          2.0|(7,[2],[1.0])|
| 10.33|          3.0|(7,[3],[1.0])|
|  8.92|          2.0|(7,[2],[1.0])|
+------+-------------+-------------+
only showing top 5 rows



Now departure time can be added to the regression model.

### Flight duration model: Adding departure time

In [126]:
flights = spark.read.csv("flights.csv", sep=",", header=True, inferSchema=True, nullValue="NA")

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="org", outputCol="org_idx")
indexer_model = indexer.fit(flights)
flights = indexer_model.transform(flights)

flights = flights.withColumn("km", round(flights.mile * 1.60934, 0)).drop("mile")

onehot = OneHotEncoder(inputCols=["org_idx"], outputCols=["org_dummy"])
onehot = onehot.fit(flights)
flights = onehot.transform(flights)

buckets = Bucketizer(splits=[0,3,6,9,12,15,18,21,24], inputCol="depart", outputCol="depart_bucket")
flights = buckets.transform(flights)

onehot = OneHotEncoder(inputCols=["depart_bucket"], outputCols=["depart_dummy"])
flights = onehot.fit(flights).transform(flights)
flights.show()

+---+---+---+-------+------+---+------+--------+-----+-------+------+-------------+-------------+-------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    km|    org_dummy|depart_bucket| depart_dummy|
+---+---+---+-------+------+---+------+--------+-----+-------+------+-------------+-------------+-------------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| null|    2.0|3465.0|(7,[2],[1.0])|          3.0|(7,[3],[1.0])|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0| 509.0|(7,[0],[1.0])|          5.0|(7,[5],[1.0])|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0| 542.0|(7,[1],[1.0])|          2.0|(7,[2],[1.0])|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|    0.0|1989.0|(7,[0],[1.0])|          3.0|(7,[3],[1.0])|
|  4|  2|  5|     AA|   325|ORD|  8.92|      65| null|    0.0| 415.0|(7,[0],[1.0])|          2.0|(7,[2],[1.0])|
|  5|  2|  1|     UA|   704|SFO|  7.98|     102|    2|    1.0| 885.0|(7,[1],[1.0])|          2.0|(7,[2],

In [130]:
assembler = VectorAssembler(inputCols=  ["km","org_dummy", "depart_dummy"], outputCol="features")
flights = assembler.transform(flights)
flights.select("km","org_dummy", "depart_dummy", "features").show(5,False)

+------+-------------+-------------+------------------------------+
|km    |org_dummy    |depart_dummy |features                      |
+------+-------------+-------------+------------------------------+
|3465.0|(7,[2],[1.0])|(7,[3],[1.0])|(15,[0,3,11],[3465.0,1.0,1.0])|
|509.0 |(7,[0],[1.0])|(7,[5],[1.0])|(15,[0,1,13],[509.0,1.0,1.0]) |
|542.0 |(7,[1],[1.0])|(7,[2],[1.0])|(15,[0,2,10],[542.0,1.0,1.0]) |
|1989.0|(7,[0],[1.0])|(7,[3],[1.0])|(15,[0,1,11],[1989.0,1.0,1.0])|
|415.0 |(7,[0],[1.0])|(7,[2],[1.0])|(15,[0,1,10],[415.0,1.0,1.0]) |
+------+-------------+-------------+------------------------------+
only showing top 5 rows



In [137]:
flight_train, flight_test = flights.randomSplit([0.8, 0.2], 13)

regression = LinearRegression(labelCol="duration").fit(flight_train)
predictions = regression.transform(flight_test)

print(RegressionEvaluator(labelCol="duration").evaluate(predictions))

# Average minutes on ground at OGG for flights departing between 21:00 and 24:00
avg_night_ogg = regression.intercept + regression.coefficients[8]
print(avg_eve_ogg)

# Average minutes on ground at OGG for flights departing between 00:00 and 03:00
avg_night_ogg = regression.intercept + regression.coefficients[8]
print(avg_night_ogg)


# Average minutes on ground at JFK for flights departing between 00:00 and 03:00
avg_night_jfk = regression.intercept + regression.coefficients[8] + regression.coefficients[3]
print(avg_night_jfk)

10.68279557023109
10.470157159054379
-2.7425333357930377
48.90395499832853


## Regularization

The coefficients quantify the effect of the coressponding features. More features imply more coefficients. Parsimonious model is the one that has the minimum required number of predictors by selecting the best subset of columns. One such approach to feature selection known as "penalized regression". The model is penalized or punished for having too many coefficients. With penalized regression an additional "regularization" or "shrinkage" term is added to the loss funtion. The regularization term can be Lasso which uses absolute value of the coefficients or Ridge which uses square of the coefficients. The strength of the regularization is determined by lambda.

Coefficients that are non zero are contributing to the model but it is unlikely that all the features are equally important for predicting. 

For Ridge regression give value of zero for elasticNetParam and use regParam to specify the strength. By using Ridge regression the coefficients are now smaller since they are shrunk. 
For Lasso regression set elasticNetParam to 1. The coefficient values can indicate the most important predictors by setting the rest to zero.

A simpler model with no significant loss in performance can be created by using these predictors.

### Flight duration model: More features!


 Adding some features might improve the model. Adding other features might make it worse. More features will always make the model more complicated and difficult to interpret.

In [190]:
flights = flights.drop("features")

onehot = OneHotEncoder(inputCols=["mon"], outputCols=["mon_dummy"])
flights = onehot.fit(flights).transform(flights)

onehot = OneHotEncoder(inputCols=["dow"], outputCols=["dow_dummy"])
flights = onehot.fit(flights).transform(flights)

assembler = VectorAssembler(inputCols=  ["km","org_dummy", "depart_dummy", "dow_dummy", "mon_dummy"], outputCol="features")
flights = assembler.transform(flights)

flight_train, flight_test = flights.randomSplit([0.8, 0.2], 13)

regression = LinearRegression(labelCol="duration").fit(flight_train)
predictions = regression.transform(flight_test)
rmse = RegressionEvaluator(labelCol="duration").evaluate(predictions)
print("The test RMSE is", rmse)
print()
coeffs = regression.coefficients
print(coeffs)

The test RMSE is 10.603426704688353

[0.07442260365703582,27.167707368706242,19.995665949017873,51.730261452801216,45.67716622066441,17.52240710954814,14.814703580321478,17.066898599484237,-13.638548949659642,0.9350467119010547,4.082754272794686,6.884410153783628,4.594492831905297,8.920085499289964,8.660386163384594,0.3366200590787332,0.05343697665105215,-0.33509170185988363,0.1804058184014835,0.2068323004067617,0.13499047028356215,-2.01346827559683,-2.5175690364624708,-2.2060718771750953,-3.578897387139923,-4.381160055596363,-4.261759154832496,-4.642295907247864,-4.371847828627821,-4.038276122473209,-2.8784616188212446,-0.8034853740251807]


With all those non-zero coefficients the model is a little hard to interpret.

### Flight duration model: Regularisation!

You can find the best value for the regularization strength using cross validation.

In [197]:
regression = LinearRegression(labelCol="duration", regParam=1, elasticNetParam=1).fit(flight_train)
rmse = RegressionEvaluator(labelCol="duration").evaluate(regression.transform(flight_test))
print("The test RMSE is", rmse)
coeffs = regression.coefficients
print(coeffs)

zero_coeff = sum([beta==0 for beta in regression.coefficients])
print("Number of coefficients equal to 0:", zero_coeff)

The test RMSE is 11.606244213337014
[0.07352946004541223,5.479348461026474,0.0,29.003106728687897,21.899509851164623,0.0,-2.4152381011777506,0.0,0.0,0.0,0.0,0.0,0.0,1.2056808785451512,1.0618468972388517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Number of coefficients equal to 0: 25


Regularisation produced a far simpler model with similar test performance.